In [ ]:
# default_exp synteny

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Synteny module

> Provides functionality to construct special synteny graph from pangenome annotation (and actual sequences).

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.


In [ ]:
#exporti
import os
import glob
import json
import itertools

import pandas as pd

from skbio.io import read as skbio_read
from skbio.metadata import IntervalMetadata
from skbio.sequence import DNA

from dnasim.IO import writeFASTA
from dnasim.simulation import inverseSequence

In [ ]:
datadir = '../../1001G/annotation'
gfadir = '../../1001G/pantograph/data'

annotationFiles = sorted(glob.glob(f'{datadir}{os.path.sep}*.gff'))
sequenceFiles = sorted(glob.glob(f'{datadir}{os.path.sep}sequences{os.path.sep}*.fasta'))
transMapFile = f'{datadir}{os.path.sep}stats{os.path.sep}TransMap.map'

In [ ]:
#Parameters
doSequences = False
doCigars = False
doUS = False
gfaFilename = 'AT_Chr1_OGOnly.gfa'
seqSuffix = '_Chr1'

In [ ]:
#export
def readTransMap(transMapFile):
    # reading and preparing transmap in pandas format
    transMap = pd.read_csv(transMapFile,delimiter='\t')
    transMap.rename(columns={'Orthogroup:':'orthogroup'},inplace=True)
    transMap['orthogroup'] = transMap.orthogroup.str.rstrip(':')
    transMap.set_index('orthogroup',inplace=True)
    return transMap

In [ ]:
#export
def generateOrder(files,priorityAccession='TIAR10'):
    ind = [idx for idx,file in enumerate(files) if priorityAccession in file][0]
    idxList = list(range(len(files)))
    del idxList[ind]
    idxList = [ind] + idxList
    return idxList

In [ ]:
#export
def getIDs(iterator):
    idList = []
    for interval in iterator:
        idList.append(interval.metadata['ID'])
    
    return idList

In [ ]:
#export
def processAccession(annotationFile,sequenceFile=None):
    accessionID = os.path.splitext(os.path.basename(annotationFile))[0]
    annotationGen = skbio_read(annotationFile, format='gff3')
    
    sequenceDict = None
    
    if sequenceFile is not None:
        sequenceDict = {}
        sequenceGen = skbio_read(sequenceFile,format='fasta')
        for seq in sequenceGen:
            sequenceDict[seq.metadata['id']] = bytearray(seq.values).decode()
    
    genes = []
    for seqID,annotation in annotationGen:
        geneInts = annotation.query(metadata={'type':'gene'})
        
        for gene in geneInts:
            geneID = gene.metadata['ID'][7:]
            orthogroup = gene.metadata['OG']
            forward = gene.metadata['strand']=='+'
            start,end = gene.bounds[0]
            
            if sequenceDict is not None:
                geneSeq = sequenceDict[seqID][start:end+1]
            else:
                geneSeq = ''
            overlaps = getIDs(annotation.query(bounds=[(start,end)],metadata={'type':'gene'}))
            genes.append([geneID,orthogroup,seqID,accessionID,forward,start,end,geneSeq,overlaps])
    
    genes = pd.DataFrame(genes,columns=['geneID','orthogroup','sequenceID','accessionID','forward','start','end','geneSeq','overlapGenes'])
    genes.sort_values(by=['sequenceID','start'],inplace=True)
        
    return accessionID,genes,sequenceDict

In [ ]:
#export
def recordSegment(name,segmentIDs,sequence=None,gfaFile=None,segmentData=None):
    segmentIDs.append(name)
    segID = len(segmentIDs)
    
    if segmentData is not None and sequence is not None:
        segmentData.append(sequence)
    
    if gfaFile is not None:
        if sequence is not None:
            gfaFile.write(f'S\t{segID}\t{sequence}\n')        
        else:
            gfaFile.write(f'S\t{segID}\t{name}\n')
    return segID

In [ ]:
#export
def addLink(links,prevPathSegment,name,forward):
    '''
    `links`: mutable
    `prevPathSegment`: mutable
    '''
    if prevPathSegment is not None:
        links[prevPathSegment].add(f'{name}\t{"+" if forward else "-"}')
    return f'{name}\t{"+" if forward else "-"}'


In [ ]:
#export
def generatePathsLinks(genes,sequences,OGList,segmentIDs,links,usCounter,doUS=True,
                       segmentData=None,gfaFile=None):
    '''
    `gfaFile`: file handle to write segments to GFA file
    `OGList`: mutable
    `links`: mutable
    `usCounter`: mutable
    
    '''
    path = []
    cigar = []
    prevEnd = 0
    prevPathSegment = None
    curSeqID = ''
    for gene in genes.iterrows():
        og = gene[1].orthogroup
        
        geneSeqID = gene[1].sequenceID
        if curSeqID != geneSeqID:
            curSeqID = geneSeqID
        
        geneStart = gene[1].start
        geneEnd = gene[1].end
        geneForward = gene[1].forward
        
        if sequences is not None:
            if geneForward:
                geneSeq = sequences[geneSeqID][geneStart:geneEnd+1]
            else:
                geneSeq = inverseSequence(sequences[geneSeqID][geneStart:geneEnd+1])
        else:
            geneSeq = ''
        
        if doUS:
        
            if sequences is not None:
                usSeq = sequences[geneSeqID][prevEnd:geneStart]
            else:
                usSeq = ''

            if len(usSeq)>0:
                isUS = True
                us = f'US{usCounter:07d}'
                usCounter += 1
            else:
                isUS = False

            if isUS:
                usID = recordSegment(us,segmentIDs,usSeq,gfaFile=gfaFile,segmentData=segmentData)
        
        if og not in OGList:
            ogID = recordSegment(og,segmentIDs,geneSeq,gfaFile=gfaFile,segmentData=segmentData)
            OGList.append(og)
        else:
            ogID = segmentIDs.index(og)+1
            
        pathAdd = [f'{ogID}{"+" if geneForward else "-"}']
        if doUS and isUS:
            pathAdd.insert(0,f'{usID}+')
            
        path.extend(pathAdd)
        
        if len(cigar)>0 and doUS and isUS:
            cigar.extend(['0M','0M']) # with previous block and between two current blocks
        else:
            cigar.append('0M') # only between current blocks or between previous and current gene
                               # without unrelated sequence (intergenic) block.
        
        if doUS and isUS:
            prevPathSegment = addLink(links,prevPathSegment,usID,True)
            links[prevPathSegment] = set()
        
        prevPathSegment = addLink(links,prevPathSegment,ogID,geneForward)
        if prevPathSegment not in links:
            links[prevPathSegment] = set()
        
        prevEnd = geneEnd+1
    
    if doUS:
        
        if sequences is not None:
            usSeq = sequences[curSeqID][prevEnd:]
        else:
            usSeq = ''

        if len(usSeq)>0:
            us = f'US{usCounter:07d}'
            usID = recordSegment(us,segmentIDs,usSeq,gfaFile=gfaFile,segmentData=segmentData)
            usCounter += 1
            path.append(f'{usID}+')
            cigar.append('0M')
            prevPathSegment = addLink(links,prevPathSegment,usID,True)

    return path,cigar,usCounter

In [ ]:
#export
def writeLinks(gfaFile,links,doCigars=True):
    for linkLeft,linksRight in links.items():
        for linkRight in linksRight:
            if doCigars:
                gfaFile.write(f'L\t{linkLeft}\t{linkRight}\t0M\n')
            else:
                gfaFile.write(f'L\t{linkLeft}\t{linkRight}\t*\n')
                
def writePath(gfaFile,AccessionID,path,cigar,doCigars):   
    if doCigars:
        cigarString = ",".join(cigar)
    else:
        cigarString = "*"
        
    gfaFile.write(f'P\t{AccessionID}\t{",".join(path)}\t{cigarString}\n')
    
def writeSegmentIDs(path,segmentIDs):
    with open(path,'w') as jsf:
        json.dump(segmentIDs,jsf)
        
def readSegmentIDs(path):
    with open(path,'r') as jsf:
        return json.load(jsf)

In [ ]:
fileOrder = generateOrder(annotationFiles)

In [ ]:
usCounter = 0
links = {}
OGList = []
segmentIDs = []

In [ ]:
accessionID,genes,sequences = processAccession(annotationFiles[0],None)

/home/pigrenok/.pyenv/versions/3.8.6/lib/python3.8/site-packages/skbio/io/registry.py:547: FormatIdentificationWarning: <_io.BufferedReader name='../../1001G/annotation/10002.gff'> does not look like a gff3 file
  warn("%r does not look like a %s file"


In [ ]:
accessionID

'10002'

In [ ]:
genes

,geneID,orthogroup,sequenceID,accessionID,forward,start,end,geneSeq,overlapGenes
19226,10002_Chr1.1,OG0001582,10002_Chr1,10002,True,775,3012,,[evm.TU.10002_Chr1.1]
19227,10002_Chr1.2,OG0001824,10002_Chr1,10002,True,4175,8538,,[evm.TU.10002_Chr1.2]
19228,10002_Chr1.3,OG0001825,10002_Chr1,10002,False,9152,10443,,[evm.TU.10002_Chr1.3]
19229,10002_Chr1.4,OG0001089,10002_Chr1,10002,False,10919,14470,,[evm.TU.10002_Chr1.4]
19230,10002_Chr1.5,OG0001826,10002_Chr1,10002,True,16181,17266,,[evm.TU.10002_Chr1.5]
...,...,...,...,...,...,...,...,...,...
15212,10002_tig00002326.7,OG0000003,10002_tig00002326,10002,False,168797,172152,,[evm.TU.10002_tig00002326.7]
15213,10002_tig00002326.8,OG0000001,10002_tig00002326,10002,True,175665,176947,,[evm.TU.10002_tig00002326.8]
15214,10002_tig00002326.9,OG0000000,10002_tig00002326,10002,False,178775,179453,,[evm.TU.10002_tig00002326.9]
15215,10002_tig00002326.10,OG0000010,10002_tig00002326,10002,True,179881,181599,,[evm.TU.10002_tig00002326.10]


In [ ]:
sequences

In [ ]:
q = list(sequences.keys())
q.sort()
q

['10002_Chr1',
 '10002_Chr2',
 '10002_Chr3',
 '10002_Chr4',
 '10002_Chr5',
 '10002_ChrC',
 '10002_ChrM',
 '10002_tig00000010',
 '10002_tig00000011',
 '10002_tig00000012',
 '10002_tig00000013',
 '10002_tig00000016',
 '10002_tig00000018',
 '10002_tig00000019',
 '10002_tig00000023',
 '10002_tig00000054',
 '10002_tig00000069',
 '10002_tig00000071',
 '10002_tig00000085',
 '10002_tig00000091',
 '10002_tig00000095',
 '10002_tig00000100',
 '10002_tig00000101',
 '10002_tig00000102',
 '10002_tig00000104',
 '10002_tig00000112',
 '10002_tig00000114',
 '10002_tig00000117',
 '10002_tig00000171',
 '10002_tig00000189',
 '10002_tig00000194',
 '10002_tig00000197',
 '10002_tig00001769',
 '10002_tig00002317',
 '10002_tig00002320',
 '10002_tig00002322',
 '10002_tig00002323',
 '10002_tig00002324',
 '10002_tig00002325',
 '10002_tig00002326',
 '10002_tig00002328',
 '10002_tig00002330',
 '10002_tig00002331',
 '10002_tig00002334',
 '10002_tig00002336',
 '10002_tig00023950',
 '10002_tig00023952',
 '10002_tig0002

In [ ]:
gfaFile.close()

In [ ]:
gfaFile = open(f'{gfadir}{os.path.sep}{gfaFilename}',mode='w')
gfaFile.write('H\tVN:Z:1.0\n')

11

In [ ]:
for accessionNum in [0,1]:
    fileNum = fileOrder[accessionNum]

    accessionID,genes,sequences = processAccession(annotationFiles[fileNum],sequenceFiles[fileNum])

    seqID = f'{accessionID}{seqSuffix}'

    path,cigar,usCounter = generatePathsLinks(gfaFile,genes.loc[genes.sequenceID==seqID],sequences,
                                    OGList,segmentIDs,links,usCounter=usCounter,doUS=doUS)

    writePath(gfaFile,seqID,path,cigar,doCigars=doCigars)

In [ ]:
writeLinks(gfaFile,links,doCigars=doCigars)
gfaFile.close()
writeSegmentIDs(f'{gfadir}{os.path.sep}segName_{os.path.splitext(gfaFilename)[0]}.json',segmentIDs)

In [ ]:
path = []
cigar = []
prevEnd = 0
for gene in genes.iterrows():
    path.append(gene[1].orthogroup)
    gene[1].orthogroup
    gene[1].start
    gene[1].end

In [ ]:
q.metadata['id'],q.values

('10002_tig00000091',
 array([b'A', b'T', b'T', ..., b'G', b'A', b'A'], dtype='|S1'))

In [ ]:
bytearray(q.values).decode()

'GGTTCTGTTTATACGAGTATTGTAATTGGAAGAATTTTTTATATTAAAAAAAAATCAATTTTGAATCCAAGATTTTTCTTGTTCTTATAT'

In [ ]:
accessionNum = 0
annotation = skbio_read(annotationFiles[accessionNum], format='gff3')#, into=IntervalMetadata, seq_id=f'{accessionID}_Chr1')


/home/pigrenok/.pyenv/versions/3.8.6/lib/python3.8/site-packages/skbio/io/registry.py:547: FormatIdentificationWarning: <_io.BufferedReader name='../../1001G/annotation/10002.gff'> does not look like a gff3 file
  warn("%r does not look like a %s file"


In [ ]:
for ann in annotation:
    print(ann[0])

10002_Chr3
10002_ChrM
10002_tig00000091
10002_tig00002317
10002_Chr5
10002_Chr2
10002_tig00000095
10002_tig00000197
10002_tig00000085
10002_tig00002326
10002_tig00000112
10002_tig00002336
10002_ChrC
10002_tig00000104
10002_tig00000069
10002_Chr4
10002_Chr1


In [ ]:
ann[1]

89060 interval features
-----------------------
Interval(interval_metadata=<139868678246704>, bounds=[(775, 3012)], fuzzy=[(False, False)], metadata={'source': 'EVM', 'type': 'gene', 'score': '.', 'strand': '+', 'ID': 'evm.TU.10002_Chr1.1', 'Name': 'EVM%20prediction%2010002_Chr1.1', 'OG': 'OG0001582', 'AT': 'AT1G03370'})
Interval(interval_metadata=<139868678246704>, bounds=[(775, 3012)], fuzzy=[(False, False)], metadata={'source': 'EVM', 'type': 'mRNA', 'score': '.', 'strand': '+', 'ID': 'evm.model.10002_Chr1.1', 'Parent': 'evm.TU.10002_Chr1.1', 'Name': 'EVM%20prediction%2010002_Chr1.1'})
...
Interval(interval_metadata=<139868678246704>, bounds=[(29717819, 29717869)], fuzzy=[(False, False)], metadata={'source': 'EVM', 'type': 'exon', 'score': '.', 'strand': '+', 'ID': 'evm.model.10002_Chr1.6690.exon3', 'Parent': 'evm.model.10002_Chr1.6690'})
Interval(interval_metadata=<139868678246704>, bounds=[(29717819, 29717869)], fuzzy=[(False, False)], metadata={'source': 'EVM', 'type': 'CDS', 'sc

In [ ]:
seqID = f'{accessionID}_Chr1'

# This is needed only for processing one sequence. 
# If we process the whole accession, we will go sequence by sequence.
for seq in sequenceGen:
    if seq.metadata['id']==seqID:
        sequence = bytearray(seq.values).decode()
        annotation = skbio_read(annotationFiles[accessionNum], format='gff3', 
                                into=IntervalMetadata, seq_id=seq.metadata['id'])


# annotation = skbio_read(annotationFiles[accessionNum], format='gff3', into=IntervalMetadata, seq_id=seqID)


/home/pigrenok/.pyenv/versions/3.8.6/lib/python3.8/site-packages/skbio/io/registry.py:547: FormatIdentificationWarning: <_io.BufferedReader name='../../1001G/annotation/10002.gff'> does not look like a gff3 file
  warn("%r does not look like a %s file"


In [ ]:
geneInts = annotation.query(metadata={'type':'gene'})

In [ ]:
genes = []
for gene in geneInts:
    geneID = gene.metadata['ID'][7:]
    orthogroup = gene.metadata['OG']
    forward = gene.metadata['strand']=='+'
    start,end = gene.bounds[0]
    geneSeq = sequence[start:end+1]
    overlaps = getIDs(annotation.query(bounds=[(start,end)],metadata={'type':'gene'}))
    genes.append([geneID,orthogroup,seqID,accessionID,forward,start,end,geneSeq,overlaps])
print(len(genes))
genes = pd.DataFrame(genes,columns=['geneID','orthogroup','sequenceID','accessionID','forward','start','end','geneSeq','overlapGenes'])
genes.sort_values(by='start',inplace=True)
genes

6690


,geneID,orthogroup,sequenceID,accessionID,forward,start,end,geneSeq,overlapGenes
0,10002_Chr1.1,OG0001582,10002_Chr1,10002,True,775,3012,GCAGTGATCATGGCATCAAAGCGCATGGAGATGGTTGGTTGCTAAC...,[evm.TU.10002_Chr1.1]
1,10002_Chr1.2,OG0001824,10002_Chr1,10002,True,4175,8538,ATGATGAAGAAGGGGAAAGGAAAGAACAGTGGCTTGTTACCGAATT...,[evm.TU.10002_Chr1.2]
2,10002_Chr1.3,OG0001825,10002_Chr1,10002,False,9152,10443,TCAACTCCTCCTTGTGGCAACAACACGCAAACCTGTGGCGTAGGGC...,[evm.TU.10002_Chr1.3]
3,10002_Chr1.4,OG0001089,10002_Chr1,10002,False,10919,14470,TTAGATTTTGAAATGGTGTAACTTAGGAGCATCGAGCGTTTTTGAG...,[evm.TU.10002_Chr1.4]
4,10002_Chr1.5,OG0001826,10002_Chr1,10002,True,16181,17266,ATGAACACCATCGTCGTTGCTCAGTTGCAGAGACAATTTCAAGACT...,[evm.TU.10002_Chr1.5]
...,...,...,...,...,...,...,...,...,...
6685,10002_Chr1.6686,OG0007134,10002_Chr1,10002,False,29702914,29706729,TCAAGTTGAAATAGCCAAAACAAGATGTGAGAATTTCAAGAAATTT...,[evm.TU.10002_Chr1.6686]
6686,10002_Chr1.6687,OG0007135,10002_Chr1,10002,True,29708168,29709148,ATGGAGAAAGGGGTTGGATTTGAGAAAGACATGAAGACAGTGAGTG...,[evm.TU.10002_Chr1.6687]
6687,10002_Chr1.6688,OG0007136,10002_Chr1,10002,False,29709591,29712117,TTAGCTTTTGCAAAACAAACCTGCACGAACACGAAAGAGATGCTTC...,[evm.TU.10002_Chr1.6688]
6688,10002_Chr1.6689,OG0002082,10002_Chr1,10002,True,29714744,29715904,ATGGAGAGATCATCATCATCATCATCATCATCATCAACATCATCAT...,[evm.TU.10002_Chr1.6689]


In [ ]:
path = []
cigar = []
prevEnd = 0
for gene in genes.iterrows():
    path.append(gene[1].orthogroup)
    gene[1].orthogroup
    gene[1].start
    gene[1].end